In [1]:

import pandas as pd

# Creamos y procesamos las tablas necesarias para FastAPI

## Abrimos tablas procesadas anteriormente

- user_review_NLP.parquet

In [2]:
# Direccion del archivo comprimido y reconvertido
user_reviews_content = '../_data/02_user_reviews_NLP.parquet'

# Cargar directamente el archivo comprimido en un DataFrame
user_reviews_dataset = pd.read_parquet(user_reviews_content)

- users_items_final.parquet

In [3]:
# Direccion del archivo comprimido y reconvertido
users_items_content = '../_data/02_users_items_final.parquet'

# Cargar directamente el archivo comprimido en un DataFrame
users_items_dataset = pd.read_parquet(users_items_content)

- steam_games_final.parquet

In [4]:
# Direccion del archivo comprimido y reconvertido
steam_games_content = '../_data/02_steam_games_FastAPI.parquet'

# Cargar directamente el archivo comprimido en un DataFrame
steam_games_dataset = pd.read_parquet(steam_games_content)

## Trabajamos armando una tabla combinando los datos de las tablas

### A continuacion utilizamos: 

- 'app_name', 'id' y 'release_year' de steam_games_dataset 
- 'item_id, 'recommend' y 'sentiment_analysis' de user_reviews_dataset

Esta tabla servirá para las funciones de FastAPI:
- UsersRecommend
- UsersNotRecommend
- sentiment_analysis

Primero renombramos 'id' de steam_games_dataset para que se corresponda con 'item_id' de user_reviews_dataset. 

In [5]:
steam_games_dataset.rename(columns={'id': 'item_id'}, inplace=True)

In [6]:
# Fusionar los DataFrames por las columnas mencionadas
merged_steam_rev_data = pd.merge(steam_games_dataset[['app_name', 'item_id', 'release_year']],
                                  user_reviews_dataset[['item_id', 'recommend', 'sentiment_analysis']],
                                  on='item_id')

# Ordenar por la columna 'app_name'
merged_steam_rev_data.sort_values('release_year', inplace=True)


In [7]:
merged_steam_rev_data.head(2)

,app_name,item_id,release_year,recommend,sentiment_analysis
28818,Sword of the Samurai,327950,1989,True,2
28817,Sword of the Samurai,327950,1989,True,2


In [8]:
merged_steam_rev_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 122540 entries, 28818 to 49064
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   app_name            122540 non-null  object
 1   item_id             122540 non-null  int64 
 2   release_year        122540 non-null  object
 3   recommend           122540 non-null  bool  
 4   sentiment_analysis  122540 non-null  int64 
dtypes: bool(1), int64(2), object(2)
memory usage: 4.8+ MB


In [9]:
print(f"El DataFrame tiene {merged_steam_rev_data.shape[0]} filas y {merged_steam_rev_data.shape[1]} columnas.")

El DataFrame tiene 122540 filas y 5 columnas.


In [10]:
merged_steam_rev_data.drop_duplicates(inplace=True)

In [11]:
merged_steam_rev_data['release_year'] = merged_steam_rev_data['release_year'].astype(int)

### Controlamos el nuevo DataFrame

In [12]:
merged_steam_rev_data.head(2)

,app_name,item_id,release_year,recommend,sentiment_analysis
28818,Sword of the Samurai,327950,1989,True,2
114235,Commander Keen,9180,1990,True,2


In [13]:
merged_steam_rev_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6288 entries, 28818 to 46253
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   app_name            6288 non-null   object
 1   item_id             6288 non-null   int64 
 2   release_year        6288 non-null   int64 
 3   recommend           6288 non-null   bool  
 4   sentiment_analysis  6288 non-null   int64 
dtypes: bool(1), int64(3), object(1)
memory usage: 251.8+ KB


In [14]:
print(f"El DataFrame tiene {merged_steam_rev_data.shape[0]} filas y {merged_steam_rev_data.shape[1]} columnas.")

El DataFrame tiene 6288 filas y 5 columnas.


In [15]:
# Guardar el DataFrame fusionado en un archivo .parquet
merged_steam_rev_data.to_parquet('../FastAPI/merged_steam_rev_data.parquet', index=False)

### Ahora utilizamos: 

- 'item_id', 'user_id' y 'playtime_forever' de users_items_dataset 
- 'genres' y 'release_year' de steam_games_dataset

Esta tabla servirá para las funciones de FastAPI:
- PlayTimeGenre
- UserForGenre

In [16]:
steam_games_dataset.head()

,app_name,item_id,genres,price,developer,release_year
0,Lost Summoner Kitty,761140,Action,4.99,Kotoshiro,2018
1,Lost Summoner Kitty,761140,Casual,4.99,Kotoshiro,2018
2,Lost Summoner Kitty,761140,Indie,4.99,Kotoshiro,2018
3,Lost Summoner Kitty,761140,Simulation,4.99,Kotoshiro,2018
4,Lost Summoner Kitty,761140,Strategy,4.99,Kotoshiro,2018


In [17]:
# Fusionar los DataFrames por las columnas mencionadas
merged_steam_user_data = pd.merge(users_items_dataset[['item_id','user_id', 'playtime_forever']], steam_games_dataset[['genres', 'release_year']], left_on='item_id', right_index=True)

# Ordenar por la columna 'genres'
merged_steam_user_data.sort_values('genres', ascending=False, inplace=True)

### Controlamos el nuevo DataFrame

In [18]:
merged_steam_user_data.head()

,item_id,user_id,playtime_forever,genres,release_year
4427896,35420,Yosche,0,Web Publishing,2016
2606339,35420,Ihealwithmypenis,0,Web Publishing,2016
2606632,35420,goldghost,0,Web Publishing,2016
2609933,35420,76561198056260701,0,Web Publishing,2016
2610035,35420,GoldiLoks,0,Web Publishing,2016


In [19]:
print(f"El DataFrame tiene {merged_steam_user_data.shape[0]} filas y {merged_steam_user_data.shape[1]} columnas.")

El DataFrame tiene 1657253 filas y 5 columnas.


In [20]:
nan_present = merged_steam_user_data.isna().any().any()

if nan_present:
    print("Se encontraron valores NaN en el DataFrame.")
else:
    print("No se encontraron valores NaN en el DataFrame.")

No se encontraron valores NaN en el DataFrame.


In [21]:
merged_steam_user_data.drop_duplicates(inplace=True)

In [22]:
print(f"El DataFrame tiene {merged_steam_user_data.shape[0]} filas y {merged_steam_user_data.shape[1]} columnas.")

El DataFrame tiene 1638970 filas y 5 columnas.


In [23]:
merged_steam_user_data['release_year'] = merged_steam_user_data['release_year'].astype(int)

In [24]:
merged_steam_user_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1638970 entries, 4427896 to 1289818
Data columns (total 5 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   item_id           1638970 non-null  int64 
 1   user_id           1638970 non-null  object
 2   playtime_forever  1638970 non-null  int64 
 3   genres            1638970 non-null  object
 4   release_year      1638970 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 75.0+ MB


## Randomizacion y reducción

- Viendo que la tabla creada es extensa, creamos una mas pequeña tomando filas aleatoriamente. Para que esta sea lo suficientemente liviana y manejable por FastAPI y la matriz de recomendaciones, la reducimos aproximadamente al 3%, que se asemeja al tamaño de la tabla merged_steam_rev_data.

In [25]:
frac=int(merged_steam_user_data.shape[0]*0.003) # Indice para obtener la fraccion de reduccion
merged_steam_user_data = merged_steam_user_data.sample(frac, random_state=62)  # 'random_state' para reproducibilidad

- Se guarda la nueva tabla

In [26]:
# Guardar el DataFrame fusionado en un archivo .parquet
merged_steam_user_data.to_parquet('../FastAPI/merged_steam_user_data.parquet', index=False)